# COGS 108 - Assignment 3: Data Privacy

# Important Reminders

**You must submit this file (`A3_DataPrivacy.ipynb`) to TritonED to finish the homework.**

This assignment has hidden tests - tests that are not visible here but that will be run on your submitted assignment.
- This means passing all the tests you can see in the notebook here does not guarantee you have the right answer!

Use the Tutorials notebooks as reference, as they often contain similar examples to those used in the assignment.

## Overview

We have discussed in lecture the importance and the mechanics of protecting individuals privacy when they are included in datasets. 

One method to do so is the Safe Harbor Method. The Safe Harbour method specifies how to protect individual's identities by telling us which tells us which information to remove from a dataset in order to avoid accidently disclosing personal information. 

In this assignment, we will explore web scraping, which can often include personally identifiable information, how identity can be decoded from badly anonymized datasets, and also explore using Safe Harbour to anonymize datasets properly. 

The topics covered in this assignment are mainly covered in the 'DataGathering' and 'DataPrivacy&Anonymization' Tutorial notebooks.

### Installing new packages

In the first part of the assignment we will understand how we can scrape the web for data. You have to use the Beautiful Soup library in Python for scraping the data. 

The library is not installed in Anaconda version, therefore to install a new library for Anaconda, we can use the conda package manager, which the cell below does for you. 

In [1]:
# Run this cell to install beautifulsoup4
#  You only need to do the installation once
#    Once you have run it you can comment these two lines so that the cell doesn't execute everytime.
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
# Imports - these provided for you. Do not import any other packages
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

## Part 1: Web Scraping 

### Scraping Rules

1) If you are using another organizations website for scraping, make sure to check the website's terms & conditions. 

2) Do not request data from the website too aggressively (quickly) with your program (also known as spamming), as this may break the website. Make sure your program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.

3) The layout of a website may change from time to time. Because of this, if you're scraping website, make sure to revisit the site and rewrite your code as needed.

### 1a) Web Scrape

We will first retrieve the contents on a page and examine them a bit.

Make a variable called 'wiki', that stores the following URL (as a string):
'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population'

To open the URL you can use 'requests.get' as shown in the cell above. Call this variable 'page'
After that use BeautifulSoup Library to open the URL and assign it to an object called 'soup'

In [3]:
wiki = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population'
page = requests.get(wiki)
soup = BeautifulSoup(page.content)

In [4]:
assert wiki
assert page
assert soup


### 1b) Checking Scrape Contents

Extract the title from the page and save it in a variable called `title_page`. 
Make sure you extract it as a string.
To do so, you have to use the soup object created in the above cell. 
Hint: from your soup variable, you can access this with `.title.string`
Make sure you print out and check the contents of `title_page`. 
Note that it should not have any tags (such as `<title>` included in it).


In [5]:
title_page = soup.title.string
title_page

'List of states and territories of the United States by population - Wikipedia'

In [6]:
assert title_page
assert isinstance(title_page, str)


### 1c) Extracting Tables

In order to extract the data we want, we'll start with extracting a data table of interest. 
Note that you can see this table by going to look at the link we scraped.
Use the soup object and call a method called find, which will and extract the first table in scraped webpage. 
Note: you need to search for the name `table`, and set the `class_` argument as `wikitable sortable`.


In [7]:
right_table = soup.find('table', class_="wikitable sortable")

In [8]:
assert right_table
assert isinstance(right_table, bs4.element.Tag)
assert right_table.name == 'table'

Extract the data from the table into lists.
Note: This code provided for you. Do read through it and try to see how it works.


In [9]:
lst_a, lst_b, lst_c = [], [], []

for row in right_table.findAll('tr'):
    
    cells = row.findAll('td')
    
    # Skips rows that aren't 10 columns long (like the heading)
    if len(cells) != 11:
        continue

    # This catches when the name cells stops having a link
    #  and ends, skipping the last (summary rows)
    try:
        lst_a.append(cells[2].find('a').text.strip())
        lst_b.append(cells[3].find(text=True).strip())
        lst_c.append(cells[4].find(text=True).strip())
    except:
        break

### 1d) Collecting into a dataframe

Create a dataframe `my_df` and add the data from the lists above to it. 
- `lst_a` is the state or territory name. Set the column name as `State`, and make this the index
- `lst_b` is the population estimate. Add it to the dataframe, and set the column name as `Population Estimate`
- `lst_c` is the census population. Add it to the dataframe, and set the column name as `Census Population`


In [10]:
af = {'State': lst_a}
my_df = pd.DataFrame(data=af)
my_df['Population Estimate'] = lst_b
my_df['Census Population'] = lst_c
my_df.set_index('State', inplace=True)
my_df

,Population Estimate,Census Population
State,,
California,"39,557,045","37,252,895"
Texas,"28,701,845","25,146,105"
Florida,"21,299,325","18,804,623"
New York,"19,542,209","19,378,087"
Pennsylvania,"12,807,060","12,702,887"
Illinois,"12,741,080","12,831,549"
Ohio,"11,689,442","11,536,725"
Georgia,"10,519,475","9,688,681"
North Carolina,"10,383,620","9,535,692"


In [11]:
assert isinstance (my_df, pd.DataFrame)
assert my_df.index.name == 'State'
assert list(my_df.columns) == ['Population Estimate', 'Census Population']


### 1e) Using the data
What is the Population Estimate of California? Save this answer to a variable called `calif_pop`
Notes:
- Extract this value programmatically from your dataframe (as in, don't set it explicitly, as `cf = 123`)
- You can use `.loc` to extract a particular value from a dataframe.
- The data in your dataframe will be strings - that's fine, leave them as strings (don't typecast).


In [12]:
calif_pop = my_df.loc['California', 'Population Estimate']
calif_pop

'39,557,045'

In [13]:
assert calif_pop


## Part 2: Identifying Data

Data Files:
- anon_user_dat.json
- employee_info.json

You will first be working with a file called `anon_user_dat.json`. This file that contains information about some (fake) Tinder users. When creating an account, each Tinder user was asked to provide their first name, last name, work email (to verify the disclosed workplace), age, gender, phone # and zip code. Before releasing this data, a data scientist cleaned the data to protect the privacy of Tinder's users by removing the obvious personal identifiers: phone #, zip code, and IP address. However, the data scientist chose to keep each users' email addresses because when they visually skimmed a couple of the email addresses none of them seemed to have any of the user's actual names in them. This is where the data scientist made a huge mistake!

We will take advantage of having the work email addresses by finding the employee information of different companies and matching that employee information with the information we have, in order to identify the names of the secret Tinder users!

### 2a) Load in the 'cleaned' data 

Load the json file into a pandas dataframe. Call it `df_personal`.

In [14]:
df_personal = pd.DataFrame()
df_personal = pd.read_json('anon_user_dat.json')
df_personal

,age,email,gender
0,60,gshoreson0@seattletimes.com,Male
1,47,eweaben1@salon.com,Female
2,27,akillerby2@gravatar.com,Male
3,46,gsainz3@zdnet.com,Male
4,72,bdanilewicz4@4shared.com,Male
5,46,sdeerness5@wikispaces.com,Female
6,53,jstillwell6@ustream.tv,Female
7,29,mpriestland7@opera.com,Male
8,39,nerickssen8@hatena.ne.jp,Female
9,45,hparsell9@xing.com,Male


In [15]:
assert isinstance(df_personal, pd.DataFrame)


### 2b) Check the first 10 emails 

Save the first 10 emails to a Series, and call it `sample_emails`. 
You should then and print out this Series. 
The purpose of this is to get a sense of how these work emails are structured and how we could possibly extract where each anonymous user seems to work.


In [16]:
sample_emails = pd.Series()
i = 0
for i in range(10):
    sample_emails.at[i] = df_personal.at[i, 'email']

print(sample_emails)

0    gshoreson0@seattletimes.com
1             eweaben1@salon.com
2        akillerby2@gravatar.com
3              gsainz3@zdnet.com
4       bdanilewicz4@4shared.com
5      sdeerness5@wikispaces.com
6         jstillwell6@ustream.tv
7         mpriestland7@opera.com
8       nerickssen8@hatena.ne.jp
9             hparsell9@xing.com
dtype: object


In [17]:
assert isinstance(sample_emails, pd.Series)


### 2c) Extract the Company Name From the Email 

Create a function with the following specifications:
- Function Name: extract_company
- Purpose: to extract the company of the email (i.e., everything after the @ sign but before the .)
- Parameter(s): email (string)
- Returns: The extracted part of the email (string)
- Hint: This should take 1 line of code. Look into the find('') method. 

You can start with this outline:
```python 
def extract_company(email):
    return
```

Example Usage: 
- extract_company("larhe@uber.com") should return "uber"
- extract_company(“ds@cogs.edu”) should return “cogs”



In [18]:
def extract_company(email):
    return email[(email.find('@')+1):email.find('.')]

test = "gshoreson0@seattletimes.com"
extract_company(test)

'seattletimes'

In [19]:
assert extract_company("gshoreson0@seattletimes.com") == "seattletimes"


With a little bit of basic sleuthing (aka googling) and web-scraping (aka selectively reading in html code) it turns out that you've been able to collect information about all the present employees/interns of the companies you are interested in. Specifically, on each company website, you have found the name, gender, and age of its employees. You have saved that info in employee_info.json and plan to see if, using this new information, you can match the Tinder accounts to actual names.

### 2d) Load in employee data 

Load the json file into a pandas dataframe. Call it `df_employee`.

In [20]:
df_employee = pd.DataFrame()
df_employee = pd.read_json('employee_info.json')
df_employee

,age,company,first_name,gender,last_name
0,42,123-reg,Inglebert,Male,Falconer
1,14,163,Rafael,Male,Bedenham
2,31,163,Lemuel,Male,Lind
3,45,163,Penny,Female,Pennone
4,52,163,Elva,Female,Crighton
5,55,1688,Herminia,Female,Sisse
6,30,1und1,Toby,Female,Nisuis
7,36,1und1,Kylynn,Female,Vedikhov
8,37,1und1,Mychal,None,Denison
9,32,360,Angelle,Female,Kupisz


In [21]:
assert isinstance(df_employee, pd.DataFrame)


### 2e) Match the employee name with company, age, gender 

Create a function with the following specifications:
- Function name: employee_matcher
- Purpose: to match the employee name with the provided company, age, and gender
- Parameter(s): company (string), age (int), gender (string)
- Returns: The employee first_name and last_name like this: return first_name, last_name 
- Note: If there are multiple employees that fit the same description, first_name and last_name should return a list of all possible first names and last name i.e., ['Desmund', 'Kelby'], ['Shepley', 'Tichner']

Hint:
There are many different ways to code this. An unelegant solution is to loop through `df_employee` 
   and for each data item see if the company, age, and gender match
   i.e., 
   ```python
   for i in range(0, len(df_employee)):
             if (company == df_employee.ix[i,'company']):
   ```
   
However! The solution above is very inefficient and long, so you should try to look into this:
Google the df.loc method: It extracts pieces of the dataframe
   if it fulfills a certain condition.
   i.e., 
   
```python
df_employee.loc[df_employee['company'] == company]
```

If you need to convert your pandas data series into a list, you can do ```list(result)``` where result is a pandas "series"

You can start with this outline:
```python
def employee_matcher(company, age, gender):
    return first_name, last_name
```


In [22]:
def employee_matcher(company, age, gender):
    holder = df_employee.loc[(df_employee['company'] == company) & (df_employee['age'] == age) & (df_employee['gender'] == gender)]
    first_name = list(holder.first_name)
    last_name = list(holder.last_name)
    return first_name, last_name

employee_matcher("salon", 47, "Female")

(['Elenore'], ['Gravett'])

In [23]:
assert employee_matcher("google", 41, "Male") == (['Maxwell'], ['Jorio'])
assert employee_matcher("salon", 47, "Female") == (['Elenore'], ['Gravett'])


### 2f) Extract all the private data 

- Create 2 empty lists called `first_names` and `last_names`
- Loop through all the people we are trying to identify in df_personal
- Call the `extract_company function` (i.e., `extract_company(df_personal.ix[i, 'email'])` )
- Call the `employee_matcher` function 
- Append the results of `employee_matcher` to the appropriate lists (`first_names` and `last_names`)



In [24]:
first_names = []
last_names = []
i = 0
for i in range(len(df_personal.index)):
    company = extract_company(df_personal.at[i, 'email'])
    age = df_personal.at[i, 'age']
    gender = df_personal.at[i, 'gender']
    names = employee_matcher(company, age, gender)
    first_names.append(names[0])
    last_names.append(names[1])
print(first_names[45:50])
print(last_names[45:50])

#df_personal['first_name'] = first_names
#df_personal['last_name'] = last_names
#df_personal

[['Justino'], ['Tadio'], ['Kennith'], ['Cedric'], ['Amargo']]
[['Corro'], ['Blackford'], ['Milton'], ['Yggo'], ['Grigor']]


In [25]:
assert first_names[45:50]== [['Justino'], ['Tadio'], ['Kennith'], ['Cedric'], ['Amargo']]
assert last_names[45:50] == [['Corro'], ['Blackford'], ['Milton'], ['Yggo'], ['Grigor']]


### 2g) Add the names to the original 'secure' dataset! 

We have done this last step for you below, all you need to do is run this cell.
For your own personal enjoyment, you should also print out
  the new `df_personal` with the identified people. 

In [26]:
df_personal['first_name'] = first_names
df_personal['last_name'] = last_names

We have now just discovered the 'anonymous' identities of all the registered Tinder users...awkward.

## Part 3: Anonymize Data

You are hopefully now convinced that with some seemingly harmless data a hacker can pretty easily discover the identities of certain users. Thus, we will now clean the original Tinder data ourselves according to the Safe Harbor Method in order to make sure that it has been *properly* cleaned...

### 3a) Load in personal data 

Load the user_dat.json file into a pandas dataframe. Call it `df_users`.
Note: You might find that using the same method as A2 (or above) leads to an error.
The file has a slightly different organization. 
  Try googling the error and finding the fix for it.
Hint: you can still use `pd.read_json`, you just need to add another argument.

In [27]:
df_users = pd.DataFrame()
df_users = pd.read_json('user_dat.json', lines=True)
df_users

,age,email,first_name,gender,ip_address,last_name,phone,zip
0,60,gshoreson0@seattletimes.com,Gordon,Male,230.97.219.70,DelaField,(245)663-9899,6705
1,47,eweaben1@salon.com,Elenore,Female,202.253.80.173,Gravett,(445)749-6402,40330
2,27,akillerby2@gravatar.com,Abbe,Male,15.120.128.79,Stockdale,(312)997-8947,44139
3,46,gsainz3@zdnet.com,Guido,Male,71.234.147.178,Comfort,(530)815-7417,44201
4,72,bdanilewicz4@4shared.com,Brody,Male,68.192.188.136,Pinckard,(214)138-6981,72956
5,46,sdeerness5@wikispaces.com,Shalne,Female,204.227.6.124,Smail,(918)972-0284,53086
6,53,jstillwell6@ustream.tv,Joell,Female,37.146.221.194,Bowlesworth,(113)112-6771,52073
7,29,mpriestland7@opera.com,Manfred,Male,67.64.181.77,Bricket,(665)414-5000,92617
8,39,nerickssen8@hatena.ne.jp,Neille,Female,180.183.192.79,McCahey,(218)925-2761,55027
9,45,hparsell9@xing.com,Henri,Male,32.181.36.170,Scotchford,(433)404-7429,74647


In [28]:
assert isinstance(df_users, pd.DataFrame)


### 3b) Drop personal attributes 

Remove any personal information, following the Safe Harbour method.
Based on the Safe Harbour method, remove any columns from `df_users` that contain personal information. Note that details on the Safe Harbour method are covered in the Tutorials.



In [29]:
df_users.drop(['email', 'first_name', 'ip_address', 'last_name', 'phone'], axis=1, inplace=True)
df_users

,age,gender,zip
0,60,Male,6705
1,47,Female,40330
2,27,Male,44139
3,46,Male,44201
4,72,Male,72956
5,46,Female,53086
6,53,Female,52073
7,29,Male,92617
8,39,Female,55027
9,45,Male,74647


In [30]:
assert len(df_users.columns) == 3


### 3c) Drop ages that are above 90 

Safe Harbour rule C: Drop all the rows which have age greater than 90 from `df_users`.

In [31]:
df_users = df_users[df_users.age <= 90]
df_users

,age,gender,zip
0,60,Male,6705
1,47,Female,40330
2,27,Male,44139
3,46,Male,44201
4,72,Male,72956
5,46,Female,53086
6,53,Female,52073
7,29,Male,92617
8,39,Female,55027
9,45,Male,74647


In [32]:
assert df_users.shape == (993, 3)


### 3d) Load in zip code data 

Load the zip_pop.csv file into a (different) pandas dataframe. Call it `df_zip`.
Note that the zip data should be read in as strings, not ints, as would be the default. 
In read_csv, use the parameter `dtype` to specify to read `zip` as str, and `population` as int.

In [33]:
df_zip = pd.DataFrame()
df_zip = pd.read_csv('zip_pop.csv', dtype={'zip':str, 'population':int})
df_zip

,zip,population
0,01001,16769
1,01002,29049
2,01003,10372
3,01005,5079
4,01007,14649
5,01008,1263
6,01009,741
7,01010,3609
8,01011,1370
9,01012,661


In [34]:
assert isinstance(df_zip, pd.DataFrame)


### 3e) Sort zipcodes into "Geographic Subdivision" 

The Safe Harbour Method applies to "Geographic Subdivisions"
  as opposed to each zipcode itself. 
Geographic Subdivision:
  All areas which share the first 3 digits of a zip code

Count the total population for each geographic subdivision
Warning: you have to be savy with a dictionary here
To understand how a dictionary works, check the section materials,
  use google and go to discussion sections!

Instructions: 
- Create an empty dictionary: ```zip_dict = {}```
- Loop through all the zip_codes in df_zip
- Create a dictionary key for the first 3 digits of a zip_code in zip_dict
- Continually add population counts to the key that contains the 
    same first 3 digits of the zip code

To extract the population you will find this code useful:

```python
population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])
```

To extract the first 3 digits of a zip_code you will find this code useful:
```python
int(str(zip_code)[:3])
```
Note: this code may take some time (many seconds, up to a minute or two) to run



In [36]:
zip_dict = {}
for zip_code in df_zip.zip:
    zipc = int(str(zip_code)[:3])
    population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])
    if zipc not in zip_dict:
        zip_dict[zipc] = population
    else:
        zip_dict[zipc].append(population)
for key in zip_dict:
    sum(zip_dict[key])
zip_dict

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [ ]:
assert isinstance(zip_dict, dict)
assert zip_dict[100] == 1502501


### 3f) Explain this code excerpt 
Note: you do not have to use this line of code at this point in the assignment.
It is one of the lines provided to you in 2e. Here, just write a quick comment on what it does. This question will not be graded, but it's important to be able to read other people's code.

In [ ]:
# In the cell below, explain in words what what the following line of code is doing:
population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])

The population variable is being set to a list that contains a population number based on the zip_code key that is being used as an iterator through df_zip.zip.

### 3g) Masking the Zip Codes 

In this part, you should write a for loop, updating the df_users dataframe.
Go through each user, and update their zip-code, to Safe Harbour specifications:
- If the user is from a zip code for the which the "Geographic Subdivision" is less than equal to 20000, change the zip code to 0 
- Otherwise, change the zip code to be only the first 3 numbers of the full zip cide
- Do all this re-writting the zip_code columns of the `df_users` DataFrame

Hints: This will be several lines of code, looping through the DataFrame, getting each zip code, checking the geographic subdivision with the population in `zip_dict`, and settig the `zip_code` accordingly. 

In [ ]:
assert len(df_users) == 993
assert sum(df_users.zip == 0) == 7
assert df_users.loc[671, 'zip'] == 359


### 3h) Save out the properly anonymized data to json file 

Save out df_users as a json file, called `real_anon_user_dat.json`

In [37]:
df_users.to_json('real_anon_user_dat.json')

In [38]:
assert isinstance(pd.read_json('real_anon_user_dat.json'), pd.DataFrame)

Congrats, you're done! The users identities are much more protected now. 

Submit this notebook file to TritonED.